In [4]:
import telegram
import xml.etree.ElementTree as et
import json
import requests as req

In [5]:
from telegram import Update
from telegram.ext import Updater, CallbackContext
from telegram.ext import MessageHandler
from telegram.ext import CommandHandler
from telegram.ext import Filters

In [6]:
mykey = '5925683914:AAHodNMQ2UQgahXBusavtKQbyIjaQXy_Rsc'
chat_id = 5797700110

## 코로나 확진자 정보 출력
* /reload 명령 입력시 관련정보를 서버에서 받아서 파일로 저장해 둠
* 이렇게 함으로써 사용자에게 나름 최신정보를 제공

### 서버에서 미세정보 파일 내려받기

In [4]:
# 서버에서 코로나 정보 내려받기
url = 'http://apis.data.go.kr/1352000/ODMS_COVID_04/callCovid04Api'
params = {'serviceKey':'GjgjTYLP10LmklhSDVDE6qQ8KZPc43DI+J0iaotwuQpc5cFzNY19Uo269sO/5MQJ+GT6AmPGbn3prlagN3nf7A==',
         'numOfRows':1000,
         'std_day':'2022-11-24'}
res = req.get(url, params=params)

In [5]:
with open('data/covid.xml','w', encoding='utf-8') as f:
    f.writelines(res.text)    

### 지역정보 입력시 코로나 정보 출력

In [6]:
# xml로 저장된 파일의 내용 읽어오기
tree = et.parse('data/covid.xml')
root = tree.getroot()

In [7]:
sido = '서울'

for data in root.findall('body/items/item'):
    sidoName = data.find('gubun').text
    
    if sidoName == sido:
        inc_dec = data.find('incDec').text
        def_cnt = data.find('defCnt').text
        date = data.find('stdDay').text
        break
        
print(sidoName, inc_dec, int(def_cnt)+int(inc_dec), date)


서울 11456 5235561 2022-11-24


### 코로나 현황 챗봇 구현 (xml)

In [7]:
import datetime as dt

str(dt.datetime.now())[:10]

'2022-11-24'

In [8]:
import datetime as dt
# data.go.kr에서 대기정보 받아서 파일로 저장
url = 'http://apis.data.go.kr/1352000/ODMS_COVID_04/callCovid04Api'
params = {'serviceKey':'GjgjTYLP10LmklhSDVDE6qQ8KZPc43DI+J0iaotwuQpc5cFzNY19Uo269sO/5MQJ+GT6AmPGbn3prlagN3nf7A==',
         'numOfRows':1000,
         'std_day':str(dt.datetime.now())[:10]} 
def reload_covid():
    res = req.get(url,params=params)
    with open('data/covid.xml','w', encoding='utf-8') as f:
        f.writelines(res.text)    

In [16]:
# 특정지역의 대기정보와 이모지 받아오기
def fetch_covid(sido):
    tree = et.parse('data/covid.xml')
    root = tree.getroot()
        
    covidinfo = '입력하신 지역에는 코로나정보가 없어요!😂😂'
    for data in root.findall('body/items/item'):
        sidoName = data.find('gubun').text
    
        if sidoName == sido:
            inc_dec = int(data.find('incDec').text)
            def_cnt = int(data.find('defCnt').text)
            date = data.find('stdDay').text
            covidinfo = f'🤕코로나확진 정보🤕\n[{sidoName} {date} ]\n증가수: {inc_dec:,d}명\n금일 확진자수: {int(def_cnt+inc_dec):,d}명'
            break
    return covidinfo

In [24]:
print(fetch_covid('서울'))

🤕코로나확진 정보🤕
[서울 2022-11-24 ]
증가수: 11,456명
금일 확진자수: 5,235,561명


### 메세지 함수 정의

In [20]:
mykey = '5925683914:AAHodNMQ2UQgahXBusavtKQbyIjaQXy_Rsc'
chat_id = 5797700110

In [18]:
def covid_message(update,context):
    info = update.message.text
    covidinfo = fetch_covid(info)
    context.bot.send_message(text=f'{covidinfo}',
                   chat_id=update.message.chat_id)

In [19]:
def reload_message(update,context):
    reload_covid()
    context.bot.send_message(text=f'🎉코로나정보 갱신됨!!🎉',
                   chat_id=update.message.chat_id)


### 핸들러 등록

In [21]:
updater = Updater(mykey)
reload_handler = CommandHandler('reload',reload_message)
covid_handler = MessageHandler(Filters.text,covid_message)

updater.dispatcher.add_handler(reload_handler)
updater.dispatcher.add_handler(covid_handler)

### 챗봇 서버 가동

In [22]:
print('챗봇 서버 가동중...')
updater.start_polling()
updater.idle()

챗봇 서버 가동중...
